# LinkedIn bot

This personal project is a bot that simulates a certain number of actions on the website linkedin.com.

The code uses DOM (Document Object Model) concepts from the HTML Layout Structure, to identify the buttons that are to be clicked on, or the free text fields that are to be filled.  

Sequentially, it logs in, searches for recruiter filtering by location, gathers the profiles in a list, then iterates through it and sends a taylored message to each individual.

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import getpass
import time
from datetime import datetime
import pandas as pd

class LinkedInBot:
    def __init__(self, username, password, driver_path):
        self.username = username
        self.password = password
        self.driver_path = driver_path
        self.browser = None

    def start_browser(self):
        service = Service(self.driver_path)
        self.browser = webdriver.Firefox(service=service)
        
    def login(self):
        self.browser.get('https://www.linkedin.com/uas/login')
        time.sleep(2)
        elementID = self.browser.find_element(By.ID, 'username')
        elementID.send_keys(self.username)
        elementID = self.browser.find_element(By.ID, 'password')
        elementID.send_keys(self.password)
        elementID.submit()
        time.sleep(4)

    def close_browser(self):
        if self.browser:
            self.browser.quit()

class RecruiterFinder:
    def __init__(self, bot, companies):
        self.bot = bot
        self.companies = companies

    def search_recruiters(self):
        for company in self.companies:
            self.search_recruiter(company)

    def search_recruiter(self, company):
        search_element = 'recruiter ' + company
        time.sleep(2)
        elementID = self.bot.browser.find_element(By.CLASS_NAME, 'search-global-typeahead__input')
        self.bot.browser.execute_script("arguments[0].click();", elementID)
        time.sleep(2)
        elementID.send_keys(Keys.BACKSPACE)
        elementID.send_keys(search_element)
        time.sleep(2)
        elementID.send_keys(Keys.ENTER)
        time.sleep(2)


        self.filter_results()

        links_current = self.get_links_from_page()
        recruiters = self.filter_recruiter_links(links_current)
        self.process_recruiters(recruiters, company)

    def filter_results(self):
        # Click on 'People' button
        time.sleep(3)
        #people_xpath = '/html/body/div[6]/div[3]/div[2]/section/div/nav/div/ul/li[1]/button'
        people_xpath = "//button[text()='People']"
        element = self.bot.browser.find_element(By.XPATH, people_xpath)
        self.bot.browser.execute_script("arguments[0].click();", element)
        time.sleep(3)

        # Click on 'Location' button and select United States
        location_xpath = '//*[@id="searchFilter_geoUrn"]'
        element = self.bot.browser.find_element(By.XPATH, location_xpath)
        self.bot.browser.execute_script("arguments[0].click();", element)
        time.sleep(2)

        united_states_xpath = '/html/body/div[6]/div[3]/div[2]/section/div/nav/div/ul/li[5]/div/div/div/div[1]/div/form/fieldset/div[1]/ul/li[1]/label'
        element = self.bot.browser.find_element(By.XPATH, united_states_xpath)
        self.bot.browser.execute_script("arguments[0].click();", element)
        time.sleep(2)

        show_results_class = 'artdeco-button--primary.ember-view.ml2'
        element = self.bot.browser.find_element(By.CLASS_NAME, show_results_class)
        self.bot.browser.execute_script("arguments[0].click();", element)
        time.sleep(5)

    def get_links_from_page(self):
        soup = BeautifulSoup(self.bot.browser.page_source, 'html.parser')
        return [link.get('href') for link in soup.findAll('a')]

    def filter_recruiter_links(self, links):
        recruiters = []
        for e in links:
            if e not in recruiters:
                if '/in/' in e:
                    if e.strip('https://www.linkedin.com/in/')[0].islower():  # Exclude 'shared connections'
                        recruiters.append(e)
        return recruiters[:1]  # Trim number of recruiters for experiment

    def process_recruiters(self, recruiters, company):
        for recruiter in recruiters:
            self.bot.browser.get(recruiter)
            time.sleep(3)
            text = self.bot.browser.find_element(By.CLASS_NAME, 'ph5.pb5').text
            recruiter_full_name = self.bot.browser.find_element(By.CLASS_NAME, 'text-heading-xlarge.inline.t-24.v-align-middle.break-words').text
            recruiter_first_name = recruiter_full_name.split(' ')[0]

            print(recruiter_full_name, company)

            self.send_message(recruiter_first_name, recruiter_full_name)

    def send_message(self, recruiter_first_name, recruiter_full_name):
        message = (f'Hi {recruiter_first_name}, \n\n'
                   f'I am looking for a full-time opportunity and thought you might be interested in my candidacy.\n'
                   f'I am happy to share my CV for your consideration.\n\n'
                   f'Best regards,\nAlbert Cousin')
        try:
            connect_xpath = "/html/body/div[6]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/button/span[text()='Connect']"
            element = self.bot.browser.find_element(By.XPATH, connect_xpath)
            self.bot.browser.execute_script("arguments[0].click();", element)
            time.sleep(2)

            send_note_xpath = '/html/body/div[3]/div/div/div[3]/button[1]/span'
            element = self.bot.browser.find_element(By.XPATH, send_note_xpath)
            self.bot.browser.execute_script("arguments[0].click();", element)
            time.sleep(2)

            elementID = self.bot.browser.find_element(By.XPATH, '//*[@id="custom-message"]')
            elementID.send_keys(message)
            time.sleep(3)
        except Exception:
            more_xpath = "//button[.//span[text()='More']]"
            elementID = self.bot.browser.find_element(By.XPATH, more_xpath)
            self.bot.browser.execute_script("arguments[0].click();", elementID)
            time.sleep(2)

            connect_xpath = f"//div[@aria-label='Invite {recruiter_full_name} to connect']"
            elementID = self.bot.browser.find_element(By.XPATH, connect_xpath)
            self.bot.browser.execute_script("arguments[0].click();", elementID)
            time.sleep(2)

            add_note_xpath = "//button[.//span[text()='Add a note']]"
            elementID = self.bot.browser.find_element(By.XPATH, add_note_xpath)
            self.bot.browser.execute_script("arguments[0].click();", elementID)
            time.sleep(2)

            elementID = self.bot.browser.find_element(By.XPATH, '//*[@id="custom-message"]')
            elementID.send_keys(message)
            time.sleep(3)

In [47]:
if __name__ == "__main__":
    print('Please enter your linkedin password:')
    #password = getpass.getpass()
    password = '1ngdphok2y'
    username = 'albert.cousin@icloud.com'
    driver_path = '/Users/albert/Pythonproject/FirstProject/geckodriver'

    bot = LinkedInBot(username, password, driver_path)
    bot.start_browser()
    bot.login()

    companies = ['Goldman Sachs', 'Deutsche Bank', 'Jp Morgan', 'Bnp Paribas', 'Société Générale']
    finder = RecruiterFinder(bot, companies)
    finder.search_recruiters()

    bot.close_browser()

Please enter your linkedin password:
Mary Gordon Utt Goldman Sachs


InvalidSessionIdException: Message: WebDriver session does not exist, or is not active
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
InvalidSessionIDError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:448:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.session@chrome://remote/content/shared/webdriver/Assert.sys.mjs:37:4
despatch@chrome://remote/content/marionette/server.sys.mjs:315:19
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20
